<a href="https://colab.research.google.com/github/Wilkingc/ddc-datascience/blob/main/Project_6_Deep_Learning_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Recognition Algorithm

Wilking Chali Alphonse
11/25/2024

## Problem Definition


This is a supervised clasification problem.

The goal is to create a ML model with Tensorflow / Keras and create a convolutional neural network to identify if an image is a cat or a dog based on multiple inputs.


## Imports

In [ ]:
import pickle
import tensorflow.keras as keras
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

## Data Collection & Path

In [ ]:
Xurl = 'https://ddc-datascience.s3.amazonaws.com/Projects/Project.6-Images/Data/X.pickle' # Training set url

In [ ]:
Yurl = 'https://ddc-datascience.s3.amazonaws.com/Projects/Project.6-Images/Data/y.pickle' # Testing set url

In [ ]:
s3 = 'https://ddc-datascience.s3.amazonaws.com/Projects/Project.6-Images/Data/dog.jpg' # Dog image to measure model performance

In [ ]:
Xdata_load = pd.read_pickle(Xurl) # Reading Pickle file with pandas

In [ ]:
Ydata_load = pd.read_pickle(Yurl)  # Reading Pickle file with pandas

## Data Cleaning

Let's make a copy of both sets so we can have a checkpoint in case we need to re-start our work

In [ ]:
X = Xdata_load.copy()
Y = Ydata_load.copy()
Y = np.array(Ydf)

In [ ]:
X_scaled = X/255.0 # Scaling X so values fall between 0 and 1

## Exploratory Data Analysis


Check values of X and Y

### X Values

In [ ]:
X_scaled.shape, Y.shape

In [ ]:
plt.imshow(X_scaled[66,:,:,0],cmap = plt.cm.gray_r )
plt.show()

### Y Values

In [ ]:
Y[66] # 0 = dog and 1 = cats

## Data Processing


* Split X and y into training and testing sets.

In [ ]:
# Split the data up in train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.25, random_state=42)

In [ ]:
X_train.shape, X_test.shape,  y_train.shape , y_test.shape

Function in pause, just in case normalization it's needed

In [ ]:
# from sklearn.preprocessing import StandardScaler

# def scale_4d_array(array):
#   """Scales a 4D NumPy array using StandardScaler.

#   Args:
#     array: The 4D NumPy array to scale.

#   Returns:
#     The scaled 4D NumPy array.
#   """

#   # Reshape the 4D array to a 2D array for efficient scaling
#   reshaped_array = array.reshape(-1, array.shape[-1])

#   # Create a StandardScaler object
#   scaler = StandardScaler()

#   # Fit the scaler to the reshaped data
#   scaler.fit(reshaped_array)

#   # Transform the data
#   scaled_array = scaler.transform(reshaped_array)

#   # Reshape the scaled data back to the original 4D shape
#   scaled_array = scaled_array.reshape(array.shape)

#   return scaled_array

In [ ]:
# X_train =scale_4d_array(X_train)
# X_test = scale_4d_array(X_test)

Let's start our convolutional sequential model and compile it

In [ ]:
model = Sequential()

# Define input layer

model.add(
  Conv2D(name = "input",input_shape = X_train.shape[1:],
         filters = 8, kernel_size = (3,3),
    activation = 'relu',
  )
)

model.add(
    MaxPooling2D(pool_size=(2,2))
)

# Define second hidden layer
model.add(
  Conv2D(filters = 8, kernel_size = (3,3), activation = 'relu',
  )
)

model.add(
  MaxPooling2D(pool_size=(2,2))
)

model.add( Flatten()
)

# Define output layer
model.add(
  Dense(name = "output",units = 1,activation = 'sigmoid',
  )
)


# Compile model
model.compile(
  optimizer = 'adam',
  loss = 'binary_crossentropy',
  metrics = ['accuracy'],
)


### Fit model

In [ ]:
X_train.shape, y_train.shape # Check set shape values

In [ ]:
# Fit model using training data
model.fit(X_train, y_train, epochs=12) ;

In [ ]:
model.summary()

#### Testing the model with testing set

In [ ]:
val_loss, val_acc = model.evaluate(X_test, y_test)
print(val_loss)
print(val_acc)

### Function to 4D

In [ ]:
import glob
import os
import cv2
import requests
import io
from PIL import Image

In [ ]:
def image4d (url):
    '''
    This function fethces an image and converts it into a numpy array
    '''
    response = requests.get(url, stream=True)
    img = Image.open(io.BytesIO(response.content)).convert('L')
    img = np.array(img)
    return img

In [ ]:
dog = image4d(s3) # Apply the function to our s3 url

In [ ]:
resized_image = cv2.resize(doggy, (100, 100))

# Reshape to 4D with a batch dimension of 1
final = resized_image.reshape(1, 100, 100, 1 )

final.shape  # Output: (1, 100, 100, 3)

In [ ]:
plt.imshow(final[0]) # Display the image after being processed
plt.show()

In [ ]:
model.predict(final) # Take a look at the prediction value of our model

Display model prediction

In [ ]:
plt.imshow(final[0])
plt.show()

predictions = model.predict(final)
predictions = predictions > 0.5

if (predictions == 1):
  pred1 = 'dog'
else:
  pred1 = 'cat'

print("Our model predict this image to be a: ", pred1)

## Communication of Results


for our convolutional neural network:
* We have an accuracy of 0.78 and a loss of 0.4
* Our model can definitely improve with more changes
* The lowest the filters for our model was the best results and lower loss value we got.
  

## **BONUS** (optional)


* Upload an image of your (or your friend's or family's) dog or cat and use your model to predict whether the image is a dog or cat.
* Hint: you'll probably need to convert the image from color to grayscale.  OpenCV, pillow, and other libraries are your friend.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# pics = '/content/drive/MyDrive/Deep Learning/cat 1.jpg'

In [ ]:
%%capture output # Capture output
!conda install -y gdown # Install Google library since we're working in Kaggle

In [ ]:
# https://drive.google.com/file/d/11i57W44MuOaIPsibk9xmNQgCwO307XPi/view?usp=drive_link #(Picture path in drive)

In [ ]:
# %%capture output
!gdown --id 11i57W44MuOaIPsibk9xmNQgCwO307XPi #Fetch image to pwd

In [ ]:
!ls -la # Make sure that the image is in our directory

In [ ]:
def bonus(filepath):
    '''
  This function takes the imported picture and resize it to an 100, 100, 1 array
    '''
    IMG_SIZE = 100
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [ ]:
cat = bonus('cat 5.webp') # Apply function to cat image

In [ ]:
type(cat), cat.shape

* Display model prediction

In [ ]:
plt.imshow(cat[0])
plt.show()

prediction = model.predict(cat)
prediction = prediction > 0.5

if (predictions == 0):
  pred = 'dog'
else:
  pred = 'cat'

print("Our model predict this image to be a: ", pred)

In [ ]:
model.predict(cat)

In [ ]:
# class_pred = (prediction > 0.5).astype("int32")
# class_pred